In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import os
import glob
import datetime
from subprocess import run
import shutil
import os
import plotly.express as px


In [18]:
path = "finbert/actu finance/"
 
index_info_save_path = 'global index'
index_info_path = f'symbol/{index_info_save_path}.csv'
symbol = pd.read_csv(index_info_path)
tickers = yf.Tickers(" ".join(symbol.Symbol.unique()))

In [19]:
index_info = {}
columns = ["Open","High","Low","Close","Volume"]
for sym in symbol.Symbol.unique():
    tmp = tickers.tickers[sym].history(period="max")
    tmp.index= tmp.index.tz_convert(None)
    tmp.index = tmp.index.date
    index_info[sym]=tmp[columns]

In [68]:
def MSE(pred, true):
    return np.mean((pred - true) ** 2)
print(pred.shape,true.shape,x.shape)
def MAE(pred, true):
    return np.mean(np.abs(pred - true))
def RSE(pred, true):
    return np.sqrt(np.sum((true - pred) ** 2)) / np.sqrt(np.sum((true - true.mean()) ** 2))

(272, 262, 1) (272, 262, 1) (272, 30, 5)


In [21]:
keys_selected_index = ["^XAX",'^GSPC','^DJI','^IXIC','^NYA','^FCHI','^RUT','^FTSE',"^VIX","^GDAXI"]

In [22]:
columns_to_drop = []
column_target = "Close"
df_date_min = "2010-01-01"

In [23]:
for key in keys_selected_index:
    df = index_info[key]
    try :
        df.index= df.index.tz_convert(None)
        idf.index = df.index.date
    except :
        pass
    df = df.drop(columns=columns_to_drop)
    df = df.rename(columns={column_target:'OT'})
    date_to_filter = [index_date for index_date in df.index if index_date>=datetime.date.fromisoformat(df_date_min)]
    tmp = df.filter(items = date_to_filter, axis=0).copy()
    tmp.index.name = 'date'
    tmp.to_csv(f"pythonProject/dataset/custom_dataset_{key}.csv")

    

In [29]:
for key in keys_selected_index:
    print(key)
    model_name="PatchTST"
    seq_len = 30
    pred_len = 60
    dm =  256
    target_type = "MS"
    df = 128
    el = 4
    nh = 32
    ll = 0
    root_path_name="./dataset/"
    data_path_name=f"custom_dataset_{key}.csv"
    model_id_name=f"test_custom_{key}"
    data_name="custom"
    random_seed=2021
    result_folder_path = f"data/current/data_{key}"
    result_archive_path = f"data/archive/data_{key}"
    data_app = f"data/app/close_{key}"
    
    if not os.path.exists(result_folder_path):
        os.makedirs(result_folder_path)

    for directory in ["pred","true","x","log",'real_prediction']:
        if not os.path.exists(f"{result_archive_path}/{directory}"):
            os.makedirs(f"{result_archive_path}/{directory}")
    
    script = f"""python3.12 run_longExp.py --random_seed {random_seed} --is_training 1 --root_path {root_path_name}"""\
        f""" --data_path {data_path_name} --model_id {model_id_name}_{seq_len}_{pred_len} --model {model_name} --data {data_name}"""\
        f""" --features {target_type} --seq_len {seq_len} --pred_len {pred_len} --enc_in 5 --freq d --label_len {ll}"""\
        f""" --e_layers {el} --n_heads {nh} --use_gpu true --batch_size 16 --d_model {dm} --d_ff {df} --dropout 0.2"""\
        f""" --fc_dropout 0 --head_dropout 0.1 --patch_len 32 --stride 2 --des 'Exp'  --train_epochs 100 --patience 5 """\
        f""" --lradj 'TST' --pct_start 0.2 --itr 1 --learning_rate 0.0001 --do_predict >logs/LongForecasting/{model_name}_{model_id_name}_{seq_len}_{pred_len}.log"""


    run(script,cwd="pythonProject",shell=True)

    pred = np.load(f"pythonProject/results/test_custom_{key}_{seq_len}_{pred_len}_PatchTST_custom_ft{target_type}_sl{seq_len}_ll{ll}_pl{pred_len}_dm{dm}_nh{nh}_el{el}_dl1_df{df}_fc1_ebtimeF_dtTrue_Exp_0/
    pred{}.npy")
    true = np.load(f"pythonProject/results/test_custom_{key}_{seq_len}_{pred_len}_PatchTST_custom_ft{target_type}_sl{seq_len}_ll{ll}_pl{pred_len}_dm{dm}_nh{nh}_el{el}_dl1_df{df}_fc1_ebtimeF_dtTrue_Exp_0/true.npy")
    x = np.load(f"pythonProject/results/test_custom_{key}_{seq_len}_{pred_len}_PatchTST_custom_ft{target_type}_sl{seq_len}_ll{ll}_pl{pred_len}_dm{dm}_nh{nh}_el{el}_dl1_df{df}_fc1_ebtimeF_dtTrue_Exp_0/x.npy")
    real_prediction = np.load(f"pythonProject/results/test_custom_{key}_{seq_len}_{pred_len}_PatchTST_custom_ft{target_type}_sl{seq_len}_ll{ll}_pl{pred_len}_dm{dm}_nh{nh}_el{el}_dl1_df{df}_fc1_ebtimeF_dtTrue_Exp_0/real_prediction.npy")
    
    src = f"pythonProject/logs/LongForecasting/{model_name}_{model_id_name}_{seq_len}_{pred_len}.log"
    dst = f"{result_folder_path}/training_log.log"

    shutil.copyfile(src, dst)
    np.save(f"{result_folder_path}/pred.npy", pred)
    np.save(f"{result_folder_path}/true.npy", true)
    np.save(f"{result_folder_path}/x.npy", x)
    np.save(f"{result_folder_path}/real_prediction.npy", x)
    
    dst = f"{result_archive_path}/log/{datetime.date.today()}.log"

    shutil.copyfile(src, dst)
    
    np.save(f"{result_archive_path}/pred/{datetime.date.today()}.npy", pred)
    np.save(f"{result_archive_path}/true/{datetime.date.today()}.npy", true)
    np.save(f"{result_archive_path}/x/{datetime.date.today()}.npy", x)
    np.save(f"{result_archive_path}/real_prediction/{datetime.date.today()}.npy", x)


    dataset = pd.read_csv(f"pythonProject/dataset/custom_dataset_{key}.csv",index_col="date")['OT']
    start_time = datetime.date.fromisoformat(dataset.index.max()) + datetime.timedelta(days=1)
    new_date_index = pd.date_range(start_time,start_time+datetime.timedelta(days=pred_len//5*7),freq='B')
    if len(new_date_index) > pred_len:
        new_date_index = new_date_index[:pred_len]
    if len(new_date_index) == pred_len-1:
        new_date_index = pd.date_range(start_time,start_time+datetime.timedelta(days=pred_len//5*7+1),freq='B')
        if len(new_date_index) == pred_len-1:
            new_date_index = pd.date_range(start_time,start_time+datetime.timedelta(days=pred_len//5*7+2),freq='B')

    date_pred = pd.Series(real_prediction[-1,:,-1],index = new_date_index.date)
    date_pred.name = "Close_pred"
    data_for_app = pd.concat([dataset, date_pred], axis = 1)

    
    if not os.path.exists(data_app):
        os.makedirs(data_app)

    data_for_app.index.name = 'date'
    data_for_app = data_for_app.rename(columns={'OT':'Close','Close_pred':'Prediction'})
    
    data_for_app.to_csv(f"{data_app}/close.csv")
    
    fig = px.line(data_for_app )
    fig.write_html("my_visualisation.html",full_html=False,div_id='plotly-figure',include_plotlyjs='directory')

    src = f"my_visualisation.html"
    dst = f"../project_chef_d_oeurvre/templates/plotly_graphe/{key}.html"
    shutil.copyfile(src, dst)



    data = []
    data.append(data_for_app['Close'].dropna().index.max())
    data.append(data_for_app['Close'].loc[data_for_app['Close'].dropna().index.max()])
    for i in [15,30,60,80]:
        data.append(data_for_app['Prediction'].dropna().values[i//7*5])
    for i in range(2,len(data)):
        data[i] = f"{(1-data[1]/data[i]):.2%}"
    data[1] = f"{data[1]:.2f}"
    index_tab = pd.DataFrame(data=np.array(data).reshape(1,-1),columns=["last update","last close","+15j","+1m","+2m","+2.5m"])

    index_tab.to_html(f"tab.html",index=False,justify="center")
    src = f"tab.html"
    dst = f"../project_chef_d_oeurvre/templates/table/tab_{key}.html"
    shutil.copyfile(src, dst)

^XAX
^GSPC
^DJI
^IXIC
^NYA
^FCHI
^RUT
^FTSE
^VIX
^GDAXI


In [188]:
data_for_app.to_csv("test.csv")

In [191]:
test = pd.read_csv("test.csv",index_col='date')

In [192]:
test

,Close
date,
2010-01-04,7326.740234
2010-01-05,7354.870117
2010-01-06,7377.700195
2010-01-07,7393.930176
2010-01-08,7425.350098
...,...
2025-07-23,18742.542969
2025-07-24,18661.189453
2025-07-25,18560.167969


In [194]:
data_for_app

date
2010-01-04     2308.419922
2010-01-05     2308.709961
2010-01-06     2301.090088
2010-01-07     2300.050049
2010-01-08     2317.169922
                  ...     
2025-07-23    19608.998047
2025-07-24    19558.201172
2025-07-25    19438.320312
2025-07-28    19197.492188
2025-07-29    19372.177734
Name: Close, Length: 3930, dtype: float64

In [195]:
pd.concat([dataset, pd.Series(real_prediction[-1,:,-1],index = new_date_index.date)], axis = 1)

,OT,0
2010-01-04,2308.419922,NaN
2010-01-05,2308.709961,NaN
2010-01-06,2301.090088,NaN
2010-01-07,2300.050049,NaN
2010-01-08,2317.169922,NaN
...,...,...
2025-07-23,NaN,19608.998047
2025-07-24,NaN,19558.201172
2025-07-25,NaN,19438.320312
2025-07-28,NaN,19197.492188


In [96]:
key = "^XAX"

for j in [1,15,28,44,66,131,262]:
    print(key,j)
    model_name="PatchTST"
    seq_len = 30
    pred_len = j
    dm =  256
    target_type = "MS"
    df = 128
    el = 4
    nh = 32
    ll = 0
    root_path_name="./dataset/"
    data_path_name=f"custom_dataset_{key}.csv"
    model_id_name=f"test_custom_{key}"
    data_name="custom"
    random_seed=2021
    result_folder_path = f"data/current/data_{key}"
    result_archive_path = f"data/archive/data_{key}"
    data_app = f"data/app/close_{key}"
    

    for directory in ["pred","true","x","training_log",'real_prediction']:
        if not os.path.exists(f"{result_folder_path}/{directory}"):
            os.makedirs(f"{result_folder_path}/{directory}")
        
    for directory in ["pred","true","x","log",'real_prediction']:
        if not os.path.exists(f"{result_archive_path}/{directory}"):
            os.makedirs(f"{result_archive_path}/{directory}")
    
    script = f"""python3.12 run_longExp.py --random_seed {random_seed} --is_training 1 --root_path {root_path_name}"""\
        f""" --data_path {data_path_name} --model_id {model_id_name}_{seq_len}_{pred_len} --model {model_name} --data {data_name}"""\
        f""" --features {target_type} --seq_len {seq_len} --pred_len {pred_len} --enc_in 5 --freq d --label_len {ll}"""\
        f""" --e_layers {el} --n_heads {nh} --use_gpu true --batch_size 16 --d_model {dm} --d_ff {df} --dropout 0.2"""\
        f""" --fc_dropout 0 --head_dropout 0.1 --patch_len 32 --stride 2 --des 'Exp'  --train_epochs 100 --patience 5 """\
        f""" --lradj 'TST' --pct_start 0.2 --itr 1 --learning_rate 0.0001 --do_predict >logs/LongForecasting/{model_name}_{model_id_name}_{seq_len}_{pred_len}.log"""


    run(script,cwd="pythonProject",shell=True)

    pred = np.load(f"pythonProject/results/test_custom_{key}_{seq_len}_{pred_len}_PatchTST_custom_ft{target_type}_sl{seq_len}_ll{ll}_pl{pred_len}_dm{dm}_nh{nh}_el{el}_dl1_df{df}_fc1_ebtimeF_dtTrue_Exp_0/pred.npy")
    true = np.load(f"pythonProject/results/test_custom_{key}_{seq_len}_{pred_len}_PatchTST_custom_ft{target_type}_sl{seq_len}_ll{ll}_pl{pred_len}_dm{dm}_nh{nh}_el{el}_dl1_df{df}_fc1_ebtimeF_dtTrue_Exp_0/true.npy")
    x = np.load(f"pythonProject/results/test_custom_{key}_{seq_len}_{pred_len}_PatchTST_custom_ft{target_type}_sl{seq_len}_ll{ll}_pl{pred_len}_dm{dm}_nh{nh}_el{el}_dl1_df{df}_fc1_ebtimeF_dtTrue_Exp_0/x.npy")
    real_prediction = np.load(f"pythonProject/results/test_custom_{key}_{seq_len}_{pred_len}_PatchTST_custom_ft{target_type}_sl{seq_len}_ll{ll}_pl{pred_len}_dm{dm}_nh{nh}_el{el}_dl1_df{df}_fc1_ebtimeF_dtTrue_Exp_0/real_prediction.npy")
    
    src = f"pythonProject/logs/LongForecasting/{model_name}_{model_id_name}_{seq_len}_{pred_len}.log"
    dst = f"{result_folder_path}/training_log/j+{pred_len}.log"

    shutil.copyfile(src, dst)
    np.save(f"{result_folder_path}/pred/j+{pred_len}.npy", pred)
    np.save(f"{result_folder_path}/true/j+{pred_len}.npy", true)
    np.save(f"{result_folder_path}/x/j+{pred_len}.npy", x)
    np.save(f"{result_folder_path}/real_prediction/j+{pred_len}.npy", real_prediction)
    
    dst = f"{result_archive_path}/log/{datetime.date.today()}.log"

    shutil.copyfile(src, dst)
    
    np.save(f"{result_archive_path}/pred/{datetime.date.today()}_j+{pred_len}.npy", pred)
    np.save(f"{result_archive_path}/true/{datetime.date.today()}_j+{pred_len}.npy", true)
    np.save(f"{result_archive_path}/x/{datetime.date.today()}_j+{pred_len}.npy", x)
    np.save(f"{result_archive_path}/real_prediction/{datetime.date.today()}_j+{pred_len}.npy", real_prediction)


^XAX 1
^XAX 15
^XAX 28
^XAX 44
^XAX 66
^XAX 131
^XAX 262


In [98]:
dataset = pd.read_csv(f"pythonProject/dataset/custom_dataset_{key}.csv",index_col="date")['OT']
start_time = datetime.date.fromisoformat(dataset.index.max()) + datetime.timedelta(days=1)

pred_lens = [1,15,28,44,66,131,262]
pred_len = pred_lens[-1]
new_date_index = pd.date_range(start_time,start_time+datetime.timedelta(days=pred_len//5*7),freq='B')
if len(new_date_index) > pred_len:
    new_date_index = new_date_index[:pred_len]
i = 0
while len(new_date_index)!= pred_len :
    i+=1
    new_date_index = pd.date_range(start_time,start_time+datetime.timedelta(days=pred_len//5*7+i),freq='B')


pred_final = np.load(f"{result_folder_path}/real_prediction/j+{pred_lens[-1]}.npy")[-1,:,-1]
test_final = np.load(f"{result_folder_path}/pred/j+{pred_lens[-1]}.npy")[-1,:,-1]
true_final = np.load(f"{result_folder_path}/true/j+{pred_lens[-1]}.npy")[-1,:,-1]
data_MSE = [MSE(test_final,true_final)] 
data_MAE = [MAE(test_final,true_final)] 
data_RSE = [RSE(test_final,true_final)] 

for pred_len in pred_lens[-2::-1]:
    real_prediction = np.load(f"{result_folder_path}/real_prediction/j+{pred_len}.npy")[-1,:,-1]
    test = np.load(f"{result_folder_path}/pred/j+{pred_len}.npy")[-1,:,-1]
    true = np.load(f"{result_folder_path}/true/j+{pred_len}.npy")[-1,:,-1]
    
    pred_final = np.concatenate((real_prediction,pred_final[pred_len:]))
    test_final = np.concatenate((test,test_final[pred_len:]))
    true_final = np.concatenate((true,true_final[pred_len:]))

    test = np.load(f"{result_folder_path}/pred/j+{pred_len}.npy")
    true = np.load(f"{result_folder_path}/true/j+{pred_len}.npy")
    
    data_MSE.append(MSE(test,true)) 
    data_MAE.append(MAE(test,true)) 
    data_RSE.append(RSE(test,true)) 

data_MSE.append(MSE(test_final,true_final)) 
data_MAE.append(MAE(test_final,true_final)) 
data_RSE.append(RSE(test_final,true_final)) 

date_pred = pd.Series(pred_final,index = new_date_index.date)
date_pred.name = "Close_pred"
data_for_app = pd.concat([dataset, date_pred], axis = 1)

if not os.path.exists(data_app):
        os.makedirs(data_app)

data_for_app.index.name = 'date'
data_for_app = data_for_app.rename(columns={'OT':'Close','Close_pred':'Prediction'})
data_for_app.to_csv(f"{data_app}/close.csv")

fig = px.line(data_for_app )
fig.write_html("my_visualisation.html",full_html=False,div_id='plotly-figure',include_plotlyjs='directory')

src = f"my_visualisation.html"
dst = f"../project_chef_d_oeurvre/templates/plotly_graphe/{key}.html"
shutil.copyfile(src, dst)

data = []
data.append(data_for_app['Close'].dropna().index.max())
data.append(data_for_app['Close'].loc[data_for_app['Close'].dropna().index.max()])

for pred_len in [1, 15, 30, 61, 91, 182, 365]:
    data.append(data_for_app['Prediction'].dropna().values[int(pred_len/7*5)])


for i in range(2,len(data)):
    data[i] = f"{(1-data[1]/data[i]):.2%}"
    data_MSE[i-2] = f"{(data_MSE[i-2]):.2f}"
    data_MAE[i-2] = f"{(data_MAE[i-2]):.2f}"
    data_RSE[i-2] = f"{(data_RSE[i-2]):.2f}"

data_MSE[-1] = f"{(data_MSE[-1]):.2f}"
data_MAE[-1] = f"{(data_MAE[-1]):.2f}"
data_RSE[-1] = f"{(data_RSE[-1]):.2f}"

data_MSE.append("MSE")
data_MAE.append("MAE")
data_RSE.append("RSE")

data_MSE = data_MSE[::-1]
data_MAE = data_MAE[::-1]
data_RSE = data_RSE[::-1]

data[1] = f"{data[1]:.2f}"

index_tab = pd.DataFrame(data=[data,data_MSE,data_MAE,data_RSE],columns=["last update","last close","+1j","+15j","+30j","+61j","+91j","182j","365j"])

index_tab.to_html(f"tab.html",index=False,justify="center")
src = f"tab.html"
dst = f"../project_chef_d_oeurvre/templates/table/tab_{key}.html"
shutil.copyfile(src, dst)

[1.3265042, 1.1909775, 0.9646043, 0.8865835, 0.794386, 0.657594, 0.25600412, 1.8155912]
['RSE', '1.82', '0.26', '0.66', '0.79', '0.89', '0.96', '1.19', '1.33']


'../project_chef_d_oeurvre/templates/table/tab_^XAX.html'

In [94]:
tmp = np.concatenate((dict_array[140],dict_array[280][140:280]))

KeyError: 140

In [95]:
index_tab

,last update,last close,+1j,+15j,+30j,+61j,+91j,182j,365j
0,2024-04-19,4834.86,-0.17%,-0.82%,0.02%,-0.65%,-0.63%,-0.80%,1.26%
1,MSE,118224.60,4203.18,25365.09,39463.57,47419.50,53262.14,75412.70,105910.36
2,MAE,298.58,50.14,123.27,151.27,168.83,182.66,222.57,279.01
3,RSE,1.82,0.26,0.66,0.79,0.89,0.96,1.19,1.33


In [77]:
index_tab

,last update,last close,+1j,+15j,+30j,+61j,+91j,182j,365j
0,2024-04-19,4834.86,-0.17%,-0.82%,0.02%,-0.65%,-0.63%,-0.80%,1.26%
1,MSE,11890001.56%,10591035.94%,10591035.94%,10591035.94%,10591035.94%,10591035.94%,10591035.94%,10591035.94%
2,MAE,11890001.56%,27901.40%,27901.40%,27901.40%,27901.40%,27901.40%,27901.40%,27901.40%
3,RSE,11890001.56%,132.65%,132.65%,132.65%,132.65%,132.65%,132.65%,132.65%


In [67]:
data

[]

In [73]:
true_final = np.load(f"{result_folder_path}/true_j+{pred_lens[-1]}.npy")[:,:,-1]

In [74]:
true_final.shape

(272, 262)